In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy import argmax
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, Convolution2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential
import cv2 as cv
import numpy as np
import os
from tqdm import tqdm
import random
from tensorflow.keras.callbacks import TensorBoard

In [13]:
CATEGORIES = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
train_ds = []

IMG_SIZE = 128
batch_size = 50

def create_training_data():
    DATADIR = "/home/karol/python_projekty/cv2/sudoku/data/train"
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            img_array = cv.imread(os.path.join(path,img))
            gray = cv.cvtColor(img_array,cv.COLOR_RGB2GRAY)
            ret,thresh = cv.threshold(gray,127,255,cv.THRESH_BINARY_INV)
            train_ds.append([thresh, class_num])

create_training_data()
print(len(train_ds))

100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 12484.27it/s]

10000


In [14]:
random.shuffle(train_ds)

In [15]:
X = []
y = []

for features,label in train_ds:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

[[[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]]]


In [16]:
data_augmentation = keras.Sequential(
    [
        layers.RandomRotation(0.08),
    ]
)

AttributeError: 'list' object has no attribute 'RandomRotation'

In [17]:
X = X/255.0

In [21]:
X.shape[1:]

(128, 128, 1)

In [18]:
layers = \
  [ Convolution2D(128, 3, 3, input_shape=X.shape[1:], activation='relu')
   , MaxPooling2D(pool_size=(2, 2))
   , Convolution2D(64, 2, 1, activation='relu')
   , MaxPooling2D(pool_size=(2, 2))
   , Convolution2D(32, 2, 1, activation='relu')
   , MaxPooling2D(pool_size=(2, 2))
   , Flatten()
   , Dense(1024, activation='relu')
   , Dropout(0.5)
   , Dense(512, activation='relu')
   , Dropout(0.5)
   , Dense(10)
   , Activation('sigmoid')
  ]

model = Sequential()
for layer in layers:
    model.add(layer)

In [19]:
callbacks = [
    keras.callbacks.ModelCheckpoint("/home/karol/python_projekty/cv2/sudoku/models/save_at_{epoch}.h5"),
]
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.fit(np.array(X), np.array(y), batch_size=32, epochs=10)

Epoch 1/10
313/313 [==============================] - 12s 37ms/step - loss: 0.3614 - accuracy: 0.8825
Epoch 2/10
313/313 [==============================] - 12s 37ms/step - loss: 0.0738 - accuracy: 0.9766
Epoch 3/10
313/313 [==============================] - 11s 36ms/step - loss: 0.0397 - accuracy: 0.9887
Epoch 4/10
313/313 [==============================] - 12s 37ms/step - loss: 0.0343 - accuracy: 0.9892
Epoch 5/10
313/313 [==============================] - 11s 37ms/step - loss: 0.0270 - accuracy: 0.9919
Epoch 6/10
313/313 [==============================] - 12s 37ms/step - loss: 0.0212 - accuracy: 0.9929
Epoch 7/10
313/313 [==============================] - 11s 36ms/step - loss: 0.0160 - accuracy: 0.9958
Epoch 8/10
313/313 [==============================] - 12s 37ms/step - loss: 0.0179 - accuracy: 0.9944
Epoch 9/10
313/313 [==============================] - 12s 37ms/step - loss: 0.0163 - accuracy: 0.9941
Epoch 10/10
313/313 [==============================] - 12s 37ms/step - loss: 0.015

In [ ]:
img = keras.preprocessing.image.load_img(
    "/home/karol/python_projekty/cv2/sudoku/data/train/2/img003-00005.png", target_size=(128,128)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
print(argmax(predictions))